In [ ]:
from purpleair.network import SensorList
import pandas as pd
import numpy as np
import time
import datetime
from datetime import date

In [ ]:
p = SensorList()  # Initialized 11,220 sensors!
# Other sensor filters include 'outside', 'useful', 'family', and 'no_child'
df = p.to_dataframe(sensor_filter='all',
                    channel='parent')

Warnow-Chacko(South-west): 10894
Urbana(South-east):94415
Urbana on Holmes:107592

In [ ]:
from purpleair.sensor import Sensor
WarnowChacko = Sensor(10894, parse_location=True)
print(WarnowChacko)
Urbana = Sensor(94415, parse_location=True)
print(Urbana)
UrbanaOnHolmes = Sensor(107592, parse_location=True)
print(UrbanaOnHolmes)

In [ ]:
df_WarnowChacko = WarnowChacko.parent.get_historical(weeks_to_get= 14, start_date = date(2021, 12, 7),
                              thingspeak_field='primary')
df_WarnowChacko

In [5]:
df_Urbana = Urbana.parent.get_historical(weeks_to_get= 14, start_date = date(2021, 12, 7),
                              thingspeak_field='primary')
df_Urbana

In [ ]:
df_UrbanaOnHolmes = UrbanaOnHolmes.parent.get_historical(weeks_to_get= 14, start_date = date(2021, 12, 7),
                              thingspeak_field='primary')
df_UrbanaOnHolmes

In [ ]:
df_WarnowChacko.created_at=pd.DatetimeIndex(df_WarnowChacko.created_at).tz_convert('US/Central')
df_WarnowChacko['created_at'] = df_WarnowChacko['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

df_Urbana.created_at=pd.DatetimeIndex(df_Urbana.created_at).tz_convert('US/Central')
df_Urbana['created_at'] = df_Urbana['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

df_UrbanaOnHolmes.created_at=pd.DatetimeIndex(df_UrbanaOnHolmes.created_at).tz_convert('US/Central')
df_UrbanaOnHolmes['created_at'] = df_UrbanaOnHolmes['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

In [ ]:
def each_fifteen(time_str):
    new_time = datetime.datetime.strptime(time_str,'%Y-%m-%d %H:%M:%S')
    new_time = new_time.replace(second = 0)
    if new_time.minute < 8:
        new_time = new_time.replace(minute = 0)
    elif new_time.minute < 23:
        new_time = new_time.replace(minute = 15)
    elif new_time.minute < 38:
        new_time = new_time.replace(minute = 30)
    elif new_time.minute < 53:
        new_time = new_time.replace(minute = 45)
    elif new_time.minute < 60:
        new_time = new_time + datetime.timedelta(hours = 1)
        new_time = new_time.replace(minute = 0)
    return str(new_time)

In [ ]:
# df_WarnowChacko.apply(each_fifteen, axis = 0)
df_WarnowChacko['new_date'] = df_WarnowChacko.apply(lambda time: each_fifteen(time['created_at']),axis=1)
df_WarnowChacko

In [ ]:
df_Urbana['new_date'] = df_Urbana.apply(lambda time: each_fifteen(time['created_at']),axis=1)
df_Urbana

In [ ]:
df_UrbanaOnHolmes['new_date'] = df_UrbanaOnHolmes.apply(lambda time: each_fifteen(time['created_at']),axis=1)
df_UrbanaOnHolmes

In [ ]:
df_WarnowChacko_date = df_WarnowChacko.drop(columns = ['created_at'])
df_WarnowChacko_date.sort_values('new_date', ascending=True)

In [ ]:
df_Urbana_date = df_Urbana.drop(columns = ['created_at'])
df_Urbana_date.sort_values('new_date', ascending=True)

In [ ]:
df_UrbanaOnHolmes_date = df_UrbanaOnHolmes.drop(columns = ['created_at'])
df_UrbanaOnHolmes_date.sort_values('new_date', ascending=True)

In [ ]:
df_WarnowChacko_date_group = df_WarnowChacko_date.groupby('new_date').agg('mean').reset_index()
df_WarnowChacko_date_group

In [ ]:
df_Urbana_date_group = df_Urbana_date.groupby('new_date').agg('mean').reset_index()
df_Urbana_date_group

In [ ]:
df_UrbanaOnHolmes_date_group = df_UrbanaOnHolmes_date.groupby('new_date').agg('mean').reset_index()
df_UrbanaOnHolmes_date_group

In [ ]:
df_WarnowChacko_date_group.to_feather('export_WarnowChacko_feather')
df_Urbana_date_group.to_feather('export_Urbana_feather')
df_UrbanaOnHolmes_date_group.to_feather('export_UrbanaOnHolmes_feather')
print('done')